Importing dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
from pymongo import MongoClient as mc
from splinter import Browser
import pandas as pd

Setting up mongo client

In [2]:
client = mc('localhost', 27017)
db = client.news_db
collection = db.items

# Mission to Mars

## Step 1: Scraping

## Nasa news

In [3]:
url = 'https://mars.nasa.gov/news/'

response = requests.get(url)
#Check if the site is accesible
print(response.status_code)

200


In [4]:
# Create BeautifulSoup object; parse with 'lxml'
src = response.content
soup = BeautifulSoup(src, 'lxml')

In [5]:
results = soup.find_all('div', class_='slide')
print(results)

[<div class="slide">
<div class="image_and_description_container">
<a href="/news/8436/why-this-martian-full-moon-looks-like-candy/">
<div class="rollover_description">
<div class="rollover_description_inner">
For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.
</div>
<div class="overlay_arrow">
<img alt="More" src="/assets/overlay-arrow.png"/>
</div>
</div>
<img alt="Why This Martian Full Moon Looks Like Candy" class="img-lazy" data-lazy="/system/news_items/list_view_images/8436_PIA23205_annotated-32x24.jpg" src="/assets/loading_320x240.png"/>
</a>
</div>
<div class="content_title">
<a href="/news/8436/why-this-martian-full-moon-looks-like-candy/">
Why This Martian Full Moon Looks Like Candy
</a>
</div>
</div>, <div class="slide">
<div class="image_and_description_container">
<a href="/news/8426/nasa-garners-7-webby-award-nominatio

In [6]:
for result in results:
    title = result.find('div', class_='content_title').a.text
    
    para = result.find('div', class_='rollover_description_inner').text
    
    print(f"Title:  {title}")
    print(f"Article Teaser:  {para}")
    print('-------------')


Title:  
Why This Martian Full Moon Looks Like Candy

Article Teaser:  
For the first time, NASA's Mars Odyssey orbiter has caught the Martian moon Phobos during a full moon phase. Each color in this new image represents a temperature range detected by Odyssey's infrared camera.

-------------
Title:  
NASA Garners 7 Webby Award Nominations

Article Teaser:  
Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.

-------------
Title:  
NASA's Opportunity Rover Mission on Mars Comes to End

Article Teaser:  
NASA's Opportunity Mars rover mission is complete after 15 years on Mars. Opportunity's record-breaking exploration laid the groundwork for future missions to the Red Planet.

-------------
Title:  
NASA's InSight Places First Instrument on Mars

Article Teaser:  
In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.

---

In [7]:
!where chromedriver

C:\Users\skq1922\.conda\envs\PythonData\chromedriver.exe
C:\ProgramData\Anaconda3\chromedriver.exe
C:\Python35\chromedriver.exe


## JPL Mars Space Images - Featured Image

In [8]:
#splinter extraction 
executable_path = {'executable_path':'chromedriver.exe' }
browser = Browser('chrome', **executable_path)

In [9]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [10]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
    
featured_image = soup.find("article", class_ = "carousel_item").get('style')

In [11]:
featured_image_text = []
featured_image_text= featured_image.split("'")
featured_image_text=featured_image_text[1].split("'")

In [12]:
featured_image_url= "https://www.jpl.nasa.gov"+featured_image_text[0]
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17046-1920x1200.jpg


# Weather in Mars

In [13]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [14]:
tweet = soup.find('p',class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").get_text()
print(tweet)

InSight sol 159 (2019-05-08) low -100.3ºC (-148.5ºF) high -21.6ºC (-6.9ºF)
winds from the SW at 4.6 m/s (10.4 mph) gusting to 15.3 m/s (34.2 mph)
pressure at 7.40 hPapic.twitter.com/rg5UKl8dLc


# Mars Facts

In [15]:
facts_url= "https://space-facts.com/mars/"

In [16]:
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [17]:
df= tables[0]
df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [18]:
html_table = df.to_html(header=False, index= False)
print(html_table)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [19]:
base= 'https://astrogeology.usgs.gov'
usgs_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(usgs_url) 
usgs_html = browser.html                                                     
usgs_soup = BeautifulSoup(usgs_html, 'html.parser')

In [20]:
hemisphere_image_urls = []                                                  
products = usgs_soup.find('div', class_='result-list')                       # finds products
hemispheres = products.find_all('div', class_='item')                        # finds hemispheres

for hemisphere in hemispheres:                                               # iterates through hemispheres
    title = hemisphere.find('div', class_='description')                                       
    #title = hemisphere.a
    title_text = title.a.text 
    title_link = title.a.get('href')
    url= base + title_link
    browser.visit(url) 
    hem_html = browser.html                                                 
    hem_soup = BeautifulSoup(hem_html, 'html.parser')                               
    
    image = hem_soup.find('div', class_='downloads').find('ul').find('li')  
    img_url = image.a['href']
    
    hemisphere_image_urls.append({'title': title_text, 'img_url': img_url})   
    
#     browser.click_link_by_text('Back')

In [21]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]